In [2]:

from sklearn import svm 
from sklearn import preprocessing
from sklearn.preprocessing import Normalizer as normalizer
import numpy as np

train_data = np.load('lab4/training_sentences.npy',allow_pickle=True)

train_labels = np.load('lab4/training_labels.npy',allow_pickle=True)

test_data=np.load('lab4/test_sentences.npy',allow_pickle=True)

test_labels=np.load('lab4/test_labels.npy',allow_pickle=True)

def normalize_data(train_data, test_data,type=None):
    if type=="l1":
        norm=normalizer(norm="l1")
        normalized_data_train = norm.transform(train_data)
        normalized_data_test= norm.transform(test_data)
    if type=="l2":
        norm=normalizer(norm="l2")
        normalized_data_train = norm.transform(train_data)
        normalized_data_test= norm.transform(test_data)
    if type=="standard":
        scaler = preprocessing.StandardScaler()
        scaler.fit(train_data)
        normalized_data_train = scaler.transform(train_data)
        
        scaler.fit(test_data)
        normalized_data_test=scaler.transform(test_data)
    return normalized_data_train, normalized_data_test
    

In [3]:
#3,4
class BagOfWords:
    def __init__(self):
        self.d={}
        self.list=[]
    def build_vocabulary(self,data):
        for sentence in data:
            for i in range(len(sentence)):
                if sentence[i] not in self.d:
                    self.d[sentence[i]]=i
                    self.list.append(sentence[i])
        return len(self.list)
    
    def get_features(self,data):
        features=np.zeros((len(data),len(self.list)))
        i=0
        for sentence in data:
            for cuv in sentence:
                if cuv in self.d:
                    features[i][self.d[cuv]]+=1
            i+=1
        return features
                
#bag=BagOfWords()
#bag.build_vocabulary(train_data)
    

In [4]:
#5
bag=BagOfWords()
bag.build_vocabulary(train_data)
train_features=bag.get_features(train_data)

test_features=bag.get_features(test_data)
normalized_data_train, normalized_data_test=normalize_data(train_features,test_features,"l2")

In [11]:
#6
svm_model = svm.SVC(C=1, kernel='linear')
svm_model.fit(normalized_data_train, train_labels)
predicted_labels_test = svm_model.predict(normalized_data_test)


In [13]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(test_labels, predicted_labels_test)
f1 = f1_score(test_labels, predicted_labels_test)

print("Accuracy:", accuracy)
print("F1 score:", f1)

feature_coef = svm_model.coef_[0]


sorted_coef_indices = feature_coef.argsort()


negative_coef = sorted_coef_indices[:10]
print("spam:", [bag.list[i] for i in negative_coef])

positive_coef = sorted_coef_indices[:-11:-1]
print("non-spam:",[bag.list[i] for i in positive_coef])

Accuracy: 0.9152173913043479
F1 score: 0.5916230366492146
spam: ['0776xxxxxxx', 'President', 'txt', 'been', 'Name', 'NEW', 'to', '86688', 'our', 'This']
non-spam: ['will', 'question', 'your', 'I', '4', 'Your', 'this', 'Thanks', 'WITH', 'ON']
